<a href="https://colab.research.google.com/github/PasanPT/FYP_SEUSL/blob/main/Ground_Remove_Volume_Estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d
import trimesh

# Load the point cloud from the file (adjust the filename as needed)
point_cloud = o3d.io.read_point_cloud("/content/capture_data.ply")

# Filter the stockpile points based on Z-value
threshold_z = 1380  # Adjust this value based on your data
stockpile_points = np.asarray(point_cloud.points)[point_cloud.points[:, 2] < threshold_z]

# Update the point cloud with filtered points
point_cloud.points = o3d.utility.Vector3dVector(stockpile_points)

# Flip the point cloud along the Z-axis
stockpile_points = np.asarray(point_cloud.points)
stockpile_points[:, 2] = -stockpile_points[:, 2]
point_cloud.points = o3d.utility.Vector3dVector(stockpile_points)

# Recalculate normals after flipping
point_cloud.estimate_normals()

# Estimate radius for ball pivoting
distances = point_cloud.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 1.5 * avg_dist

# Create a mesh from the point cloud using ball pivoting algorithm
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    point_cloud, o3d.utility.DoubleVector([radius, radius * 2])
)

# Convert the mesh to a trimesh object
tri_mesh = trimesh.Trimesh(
    np.asarray(mesh.vertices),
    np.asarray(mesh.triangles),
    vertex_normals=np.asarray(mesh.vertex_normals),
)

# Compute the convex hull of the mesh
convex_hull_mesh = tri_mesh.convex_hull

# Check if the mesh is convex
is_convex = trimesh.convex.is_convex(convex_hull_mesh)

# Compute the volume of the convex hull
volume = convex_hull_mesh.volume
volume = volume / 95823138.44 * 3  # Adjust the scaling factor if needed

print(f"Is the mesh convex? {is_convex}")
print(f"Estimated volume of the stockpile: {volume:.2f} cubic units")
